In [1]:
! pip install Bio

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   --------------- ------------------------ 1.0/2.7 MB 25.4 MB/s eta 0:00:01
   --------------- ------------------------ 1.0/2.7 MB 25.4 MB/s eta 0:00:01
   --------------- ------------------------ 1.0/2.7 MB 25.4 MB/s eta 0:00:01
   --------------- ------------------------ 1.0/2.7 MB 25.4 MB/s eta 0:00:01
   --------------- ------------------------ 1.0/2.7 MB 25.4 MB/s eta 0:00:01
   --------------- ------------------------ 1.0/2.7 MB 25.4 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 1.8 MB/s eta 0:00:00


In [4]:
import pandas as pd
data = pd.read_csv("singscore_new_code/UK65_clean_gene_tpm.txt", sep="\t")
data

,GeneName,TPM,log2_TPM
0,7SK,131488.067500,17.004583
1,A1BG,42.941644,5.457517
2,A1BG-AS1,151.389341,7.251618
3,A1CF,1.277293,1.187320
4,A2M,422.318386,8.725599
...,...,...,...
24505,ZYG11B,15.132757,4.011921
24506,ZYX,239.063682,7.907273
24507,ZZEF1,91.359184,6.529184
24508,ZZZ3,264.379261,8.051912


In [ ]:
from Bio import Entrez
Entrez.email = "jnanshi@genesilico.ai"  # NCBI requires this

# List of gene symbols to convert
gene_symbols = data["GeneName"].to_list()
species = "Homo sapiens"

# Store mappings here
entrez_ids = {}

# Query NCBI for each gene symbol
for symbol in gene_symbols:
    print(symbol)
    handle = Entrez.esearch(db="gene", term=f"{symbol}[sym] AND {species}[orgn]")
    record = Entrez.read(handle)
    if record["IdList"]:
        entrez_ids[symbol] = record["IdList"][0]
    else:
        entrez_ids[symbol] = None

# Convert dictionary → DataFrame
df = pd.DataFrame(list(entrez_ids.items()), columns=["Symbol", "EntrezID"])

# Display or save
print(df)
# Optionally save to file
df.to_csv("gene_symbol_to_entrez_UK65.csv", index=False)

KeyboardInterrupt: 

In [11]:
from Bio import Entrez
import pandas as pd
from tqdm import tqdm
import time

# Always tell NCBI who you are
Entrez.email = "jnanshi@genesilico.ai"  # NCBI requires this

# Get list of gene symbols from your data
gene_symbols = data["GeneName"].tolist()
species = "Homo sapiens"

entrez_ids = {}

print(f"\n🔍 Starting Entrez ID lookup for {len(gene_symbols)} genes...\n")

# tqdm progress bar
for symbol in tqdm(gene_symbols, desc="Processing genes", unit="gene"):
    try:
        # Query NCBI Gene database
        handle = Entrez.esearch(db="gene", term=f"{symbol}[sym] AND {species}[orgn]")
        record = Entrez.read(handle)
        handle.close()

        if record["IdList"]:
            entrez_ids[symbol] = record["IdList"][0]
        else:
            entrez_ids[symbol] = None
            print(f"⚠️  No Entrez ID found for: {symbol}")

        # Optional: small delay to avoid NCBI rate-limit
        time.sleep(0.03)

    except Exception as e:
        print(f"❌ Error fetching {symbol}: {e}")
        entrez_ids[symbol] = None
        time.sleep(0.5)

# Convert dictionary to DataFrame
df = pd.DataFrame(list(entrez_ids.items()), columns=["Symbol", "EntrezID"])

# Summary
n_found = df['EntrezID'].notna().sum()
n_missing = df['EntrezID'].isna().sum()

print(f"\n✅ Done! Found Entrez IDs for {n_found}/{len(df)} genes "
      f"({n_missing} missing).\n")

# Save to file
output_file = "gene_symbol_to_entrez_UK65.csv"
df.to_csv(output_file, index=False)
print(f"💾 Saved results to: {output_file}")



🔍 Starting Entrez ID lookup for 24510 genes...



Processing genes:   0%|          | 20/24510 [00:19<6:46:25,  1.00gene/s]


KeyboardInterrupt: 